# Neural Network in Keras -- a Case Study of the Fashion MNIST Dataset

This code is based the (more extensive) Keras classification tutorial at (https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/classification.ipynb).

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

## Import the Fashion MNIST dataset

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

The [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset contains 70,000 grayscale images -- 60,000 for training and 10,000 for testing. Each image is of size 28 pixels by 28 pixels, and shows a piece of clothes -- see the example below.

In [ ]:
train_images.shape, test_images.shape

In [ ]:
plt.figure()
plt.imshow(train_images[0], cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

Each image is represented by a 28x28 NumPy array, with pixel values ranging from 0 to 255. For example, the image above is presented as:

In [ ]:
print(train_images[0])

These clothes belong to 10 categories/classes (so this is a 10-class prediction problem rather than a binary prediction problem). These 10 classes are:

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
print(f"The image above has a target/label value of {train_labels[0]}.")
print(f"Therefore, it is an image of {class_names[train_labels[0]]}.")

Let's plot a few more for fun:

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Normalize the data

Modern NN prefers (and often requires) input data being normalized/standardized -- a topic related to how optimizers work and how regularization works. For bounded data such as this one where values range between 0 and 255, we need to normalize it so that it is within the range of 0 to 1. For this dataset, it means dividing the values by 255. It's important that the *training set* and the *testing set* be preprocessed in the same way:

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

## Build the model

Two steps: 1. Set up the layers. 2. Compile the model.

### Set up the layers

Our goal today is to have a basic feel of how to build a NN model using Keras. We will gradually learn more of the Keras package and related programming details.

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

The first layer in this network, `tf.keras.layers.Flatten`, transforms the format of the images from a two-dimensional array (of shape (28, 28)) to a one-dimensional array (of shape (784)). Think of this layer as unstacking rows of pixels in the image and lining them up. This layer has no parameters to learn; it only reformats the data.

After the pixels are flattened, the network consists of a sequence of two `tf.keras.layers.Dense` layers. These are densely connected, or fully connected, neural layers. The first `Dense` layer has 128 nodes (or neurons). The second `Dense` layer has 10 nodes. Notice that these two dense layers use different activation functions (why?).

In [ ]:
model.summary()

### Compile the model

The model's *compile* step allows us to specify the *loss function*, the *optimizer*, and the *metrics*:

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

## Train the model

In [ ]:
model.fit(train_images, train_labels, epochs=10)

As the model trains, the loss and accuracy metrics (of every epoch) are displayed. This model reaches an accuracy of about 93.0% on the training dataset.

## Evaluate accuracy

Next, compare how the model performs on the test dataset:

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('\nTest accuracy:', test_acc)

The accuracy on the test dataset, 88.86%, is only slightly less than the accuracy on the training dataset. Therefore, overfitting is not a significant concern in this model.

## Make predictions

In [ ]:
predictions = model.predict(test_images)

Here, the model has predicted the label for each image in the testing set. Let's take a look at the first prediction:

In [ ]:
predictions[0]

In [ ]:
predicted_value = class_names[predictions[0].argmax()]
ground_truth = class_names[test_labels[0]]
print(f"Our trained NN model predicts that the first test image is a(an) {predicted_value}.")
print(f"The ground truth is a(an) {ground_truth}.")

verdict = 'correct' if predicted_value==ground_truth else 'incorrect'
print(f"Therefore, this prediction is {verdict}.")

(Not required for our course) Below are codes for more visually-fancy prediction for the first 15 records:

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()